## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette, PneumaticSampleLoader
from science_jubilee.utils import Handlers
import time
import numpy as np
import pandas as pd
import logging
import sys
import json
sys.path.append('..')
import stober_synthesis_utils as stober
import usaxs_utils
import uuid

In [4]:
with open('systemconfig.json', 'rt') as f:
    config = json.load(f)

#usaxs_server_ip = config['sample_server']['ip_address']

#afl_ip = '192.168.1.4'

reaction_time = 5*60

In [5]:
FORMAT = '%(asctime)s:%(levelname)s:%(name)s:%(message)s'
logging.basicConfig(filename = 'APS_batchSynthesis_2025_03_29.log', level = logging.INFO, format = FORMAT)
logger = logging.getLogger(__name__)

In [6]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False, crash_detection = False) 

0
response in connect:  [True, True, True, True]


In [7]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json')

In [61]:
sample_table = pd.read_csv('Mesoporous_SobolBaseline_APS_BatchMode_3_29_25.csv')
#sample_table_2 = pd.read_csv('SampleTable_ReproReplicates_10000.csv')


In [62]:
sample_table

,Unnamed: 0,sample_name,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,F127_volume
0,0,SobolBatch_control_1,5dfdbe5e-3f38-44de-b9db-65c72c27f02e,495.350825,482.903414,2348.633956,2721.331679,1907.180170,2044.599948
1,1,SobolBatch_control_2,93780a39-35f5-4c85-963b-2fde66396c78,495.350825,482.903414,2348.633956,2721.331679,1907.180170,2044.599948
2,2,SobolBatch_control_3,00afc63f-86d3-41a0-9f37-8d56c6d1abfd,495.350825,482.903414,2348.633956,2721.331679,1907.180170,2044.599948
3,3,SobolBatch_1,83caa81b-c9ac-4b2f-b7b6-97a05c42f277,828.725197,715.494259,4322.187697,2101.010854,1758.191916,274.390077
4,4,SobolBatch_2,285271f3-055d-4edd-9cdd-3322ed1140ed,739.415980,217.202421,5575.832160,1553.807338,1114.764602,798.977500
...,...,...,...,...,...,...,...,...,...
81,81,SobolBatch_76,4de2e734-e313-400b-b4e5-2817f8478e1c,178.102264,483.436935,247.239871,1971.350346,2567.553040,4552.317543
82,82,SobolBatch_77,db17e40c-71da-4e97-8660-7918dee0e954,154.427788,165.795203,3574.276972,3582.069010,1169.978902,1353.452124
83,83,SobolBatch_78,f0cd21ec-eaad-4b54-aeae-d5a3bf6f5c0b,388.439717,282.341696,1445.897968,2000.454765,3792.703204,2090.162650
84,84,SobolBatch_79,50909d33-3594-4edc-b54e-5c1dcdde3094,245.405401,715.307893,3713.554608,3199.979417,2047.251681,78.500999


## Load Tools

In [17]:
syringe_10 = syringe.HTTPSyringe.from_config(2, "../../science-jubilee/src/science_jubilee/tools/configs/10cc_syringe.json")
syringe_1_1 = syringe.HTTPSyringe.from_config(1, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_1_syringe.json")
syringe_1_2 = syringe.HTTPSyringe.from_config(3, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_2_Hamiltonsyringe.json")
syringe_1_3 = syringe.HTTPSyringe.from_config(4, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_3_Hamiltonsyringe.json")
syringe_1_4 = syringe.HTTPSyringe.from_config(0, '../../science-jubilee/src/science_jubilee/tools/configs/1cc_4_syringe.json')

Syringe name:  10cc_1
{'capacity': 10000, 'empty_position': 1830, 'full_position': 1217, 'name': '10cc_1'}
Syringe name:  1cc_1
{'capacity': 1000, 'empty_position': 1845, 'full_position': 1240, 'name': '1cc_1'}
Syringe name:  1cc_2_hamilton
{'capacity': 1000, 'empty_position': 1753, 'full_position': 1120, 'name': '1cc_2_hamilton'}
Syringe name:  1cc_3_hamilton
{'capacity': 1000, 'empty_position': 1753, 'full_position': 1120, 'name': '1cc_3_hamilton'}
Syringe name:  1cc_4
{'capacity': 1000, 'empty_position': 1845, 'full_position': 1240, 'name': '1cc_4'}


In [18]:
jubilee.load_tool(syringe_10)
jubilee.load_tool(syringe_1_1)
jubilee.load_tool(syringe_1_2)
jubilee.load_tool(syringe_1_3)
jubilee.load_tool(syringe_1_4)

In [19]:
mix_syringe = syringe_10
water_syringe = syringe_1_1
ammonia_syringe = syringe_1_2
teos_syringe = syringe_1_3
surfactant_syringe = syringe_1_4

## Load water syringe with water

In [72]:
water_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [73]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1, s = 2000)

In [75]:
water_syringe.load_syringe(0, water_syringe.empty_position - 1)

Loaded syringe, remaining volume 0 uL


## Load ammonia syringe

In [38]:
#ammonia_syringe.set_pulsewidth(1420, s = 10)

In [39]:
ammonia_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [40]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1, s = 2000)

In [41]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+1, s = 10)

In [42]:
ammonia_syringe.load_syringe(850, ammonia_syringe.full_position-1)

Loaded syringe, remaining volume 850 uL


## Load TEOS syringe

In [43]:
teos_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [44]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1, s = 2000)

In [45]:
teos_syringe.load_syringe(0, teos_syringe.empty_position - 1)

Loaded syringe, remaining volume 0 uL


## load ethanol syringe

In [66]:
#jubilee.park_tool()

In [46]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


In [47]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1, s = 2000)


## Don't need to load surfactant syringe

In [48]:
surfactant_syringe.set_pulsewidth(surfactant_syringe.empty_position-1)
surfactant_syringe.load_syringe(0, surfactant_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


## Load labware


In [96]:
samples_1 = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 2)
samples_1.manual_offset([(30.8, 168.0), (121.3, 168.3), (121.3, 112.6)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [97]:
samples_2 = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 0)
samples_2.manual_offset([(31.8, 71.0), (121.8, 71.0), (121.8, 16.0)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [98]:
stocks_1 = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks_1.manual_offset([(30.1, 264.5), (121.1, 264.5), (121.1, 208.5)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [99]:
stocks_2 = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks_2.manual_offset([(171.1, 264.8), (260.8, 265.0), (260.8, 209.0)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [100]:
samples = stober.SampleWells({'samples_1':samples_1, 'samples_2':samples_2})

In [108]:
teos_stocks = stober.Stocks([stocks_1[0], stocks_1[1]], usable_volume= 17000)
ammonia_stocks = stober.Stocks([stocks_1[2]], usable_volume= 17000)
water_stocks = stober.Stocks([stocks_1[3], stocks_1[4], stocks_1[5]], usable_volume= 17000)
ethanol_stocks = stober.Stocks([stocks_1[6], stocks_1[7], stocks_1[8], stocks_1[9]], usable_volume = 17000)
f127_stocks = stober.Stocks([stocks_1[10], stocks_1[11], stocks_2[0]], usable_volume = 17000)
ctab_stocks = stober.Stocks([stocks_2[1], stocks_2[2], stocks_2[3], stocks_2[4]], usable_volume= 17000)

rinse_stocks_precursor = [stocks_2[5], stocks_2[6], stocks_2[7]]
rinse_stocks_teos = [stocks_2[8], stocks_2[9], stocks_2[10], stocks_2[11]]

## Check sample alignment

In [28]:
jubilee.deck.safe_z

68.5

In [ ]:
jubilee.pickup_tool(mix_syringe)
for loc in stocks_1:
    jubilee.safe_z_movement()
    jubilee.move_to(x = loc.x, y = loc.y)
    print(loc.y)
    jubilee.move_to(z = loc.top_+7)
    curr_well = loc
    ans = input(f'Is the tip centered over sample {curr_well}?')
    if ans == 'y':
        continue
    else:
        continue
    

In [49]:
sample_table

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,well
0,0,82c085e1-a523-40ce-9e89-4fa0bf546525,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,B2
1,1,747534b1-7370-49a5-bdc1-4fce5227f391,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,B3
2,2,e5f6e82f-0282-46c8-be77-d8914adca9b5,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,B4
3,3,c87ac840-ddde-43ab-a8e3-b5e3eb5a273d,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,C1
4,4,d034d82c-e329-484c-91cc-75ea655a1446,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,C2


In [34]:
jubilee.park_tool()


In [106]:
loc = samples['A8']
jubilee.move_to(x = loc.x, y = loc.y)
print(loc.y)
jubilee.move_to(z = loc.top_+7)

175.8614659806774


# Experiment


In [95]:
jubilee.park_tool()

## add ethanol

In [70]:
water_syringe.remaining_volume

999

In [106]:
sample_table_batch2 = sample_table.iloc[17:41]

In [107]:
len(sample_table_batch2)

24

In [ ]:
for i, row in sample_table_batch2.iterrows():


    if i %3 == 0 and i !=0:
        # swap out CTAB
        result = input('CTAB swap out required. swap ctab with fresh from the hot plate.')
        
    sample_uid = row['uuid']
    sample_name = row['sample_name']
    well = samples.get_well_for_sample(sample_uid)
    teos_volume = row['teos_volume']
    ethanol_volume = row['ethanol_volume']
    ctab_volume = row['ctab_volume']
    f127_volume = row['F127_volume']

    sample_table_select = sample_table.iloc[i:i+1]
    location_lookup_selected = {sample_uid:well}



    start_time = time.time()
    logger.info(f'Starting sample {sample_name}, {sample_uid} in well {well}')
    print(f'Starting sample {sample_name}, {sample_uid} in well {well}')
    #add ethanol
    stober.reactant_transfer(jubilee, mix_syringe, ethanol_stocks, well, ethanol_volume, 500)
    logger.info(f'Added {ethanol_volume} uL ethanol to {str(sample_uid)}') 
    
    #add water
    stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table_select, location_lookup_selected, 'water_volume', water_stocks, dwell_time = 7)
    print(f'added water')
    logger.info(f'Added water to {str(sample_uid)}') 
    #add ammonia
    stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table_select, location_lookup_selected, 'ammonia_volume', ammonia_stocks, dwell_time = 10, refill_dwell=5)
    print('added ammonia')
    logger.info(f'Added ammonia to {str(sample_uid)}') 
    # add F127
    
    stober.reactant_transfer(jubilee, surfactant_syringe, f127_stocks, well, f127_volume, 50, rinse_stocks=rinse_stocks_precursor)
    print(f'added {f127_volume} f127')
    logger.info(f'Added {f127_volume} uL F127 to {str(sample_uid)}') 
    
    stober.reactant_transfer(jubilee, surfactant_syringe, ctab_stocks, well, ctab_volume, 50, rinse_stocks=rinse_stocks_precursor)
    print(f'added {ctab_volume} uL CTAB')
    logger.info(f'Added {ctab_volume} uL ctab to {str(sample_uid)}') 
    # mix precursors
    
    stober.first_mix(jubilee, mix_syringe, 5000, location_lookup_selected, [], 3, n_rinse = 1)
    print('mixed samples')
    # add teos
    teos_add_time = stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table_select, location_lookup_selected, 'teos_volume', teos_stocks, mix_after=(8000, 5, rinse_stocks_teos), wait = False, n_rinse = 1, return_time = True)
    
    logger.info(f'Added TEOS to sample {sample_name} at {time.strftime("%H:%M", time.localtime(teos_add_time))}')
    
    print(f'Time from start to TEOS addition: {teos_add_time - start_time} s')
    
    print(f'########## Complete sample {sample_name} ############')




Starting sample SobolBatch_15, 2a9314bf-982c-4890-8572-e8f337974424 in well Well A1 form 20mLscintillation 12 Well Plate 18000 µL on slot 2
Loaded syringe, remaining volume 0 uL
breaking dispense into 1 of volume 2720.149287636392
Loaded syringe, remaining volume 0 uL
breaking dispense into 3 of volume 714.3825466351376
remaining vol:  949.0
remaining vol:  949.0
remaining vol:  949.0
added water
breaking dispense into 1 of volume 329.65084677562123
remaining vol:  949.0
added ammonia
Loaded syringe, remaining volume 0 uL
breaking dispense into 4 of volume 773.1834291480482
Rinsing 1cc_4
Loaded syringe, remaining volume 0 uL
added 3092.7337165921926 f127
Loaded syringe, remaining volume 0 uL
breaking dispense into 2 of volume 767.8976686050495
Rinsing 1cc_4
Loaded syringe, remaining volume 0 uL
added 1535.795337210099 uL CTAB
mixed samples
breaking dispense into 1 of volume 178.52317188028243
Time from start to TEOS addition: 803.2737600803375 s
########## Complete sample SobolBatch_15

CTAB swap out required. swap ctab with fresh from the hot plate. 


Starting sample SobolBatch_16, 74cf78f3-80a0-42f1-b2b4-57440d3d9296 in well Well A2 form 20mLscintillation 12 Well Plate 18000 µL on slot 2
Loaded syringe, remaining volume 0 uL
breaking dispense into 1 of volume 2262.2655518709216
